# selenium을 활용한 웹 스크래핑 실전!

### 참조: https://aytekin.tistory.com/48

### https://book.coalastudy.com/data_crawling/week3/stage3 를 대폭 수정함

In [1]:
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,356 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubun

In [2]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [3]:
#코로나로 검색
driver.get('https://search.naver.com/search.naver?query=%EC%BD%94%EB%A1%9C%EB%82%98&where=news&ie=utf8&sm=nws_hty')

# 옵션 클릭.
elem_option = driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[2]/a')
elem_option.send_keys('\n')

# # 직접입력 클릭.
elem_input = driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]')
elem_input.send_keys('\n')

naver_news_link = []

In [4]:
def select_date_naver_news(year,month,day):

    '''네이버뉴스에서 원하는 날짜의 뉴스를 검색해서 네이버뉴스 링크를 리스트로 return.'''
    # 연(year) 클릭하기
    elem_year = driver.find_element(By.XPATH,f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{year-1989}]')
    elem_year.click()

    # 월(month) 클릭하기
    elem_month = driver.find_element(By.XPATH,f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{month}]')
    elem_month.click()

    # # 날짜(day) 클릭하기
    elem_day = driver.find_element(By.XPATH,f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{day}]')
    elem_day.click()

    # 끝나는 날짜 클릭하기
    driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[1]/span[3]/a').click()

    # 연(year) 클릭하기
    elem_year = driver.find_element(By.XPATH,f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{year-1989}]')
    elem_year.click()

    # 월(month) 클릭하기
    elem_month = driver.find_element(By.XPATH,f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{month}]')
    elem_month.click()

    # # 날짜(day) 클릭하기
    elem_day = driver.find_element(By.XPATH,f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{day}]')
    elem_day.click()

    # 적용 클릭
    driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[3]/button').click()

    naver_news_link = driver.find_elements(By.LINK_TEXT, '네이버뉴스')
    links = [link.get_attribute('href') for link in naver_news_link]

    # 옵션 초기화 버튼 클릭
    driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[7]/div/div/a[1]').click()

    time.sleep(2)

    # 직접입력 클릭.
    elem_input = driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]')
    elem_input.send_keys(Keys.ENTER)

    return links

In [5]:
# 2020년 1월부터 ~ 2021 11월 현재까지 2년치 네이버 뉴스데이터 기사 크롤링

# 2020년 1월의 네이버 뉴스데이터 기사 크롤링
for year in tqdm(range(2024,2024),desc='preprocessing year'):
    for month in tqdm(range(1,2),desc='preprocessing month'):
        for day in tqdm(range(1,32),desc='preprocessing day'):
            try:
                res = select_date_naver_news(year=year,month=month,day=day)
                naver_news_link.append(res)
            except:
                pass

preprocessing month:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessing day:   0%|          | 0/31 [00:00<?, ?it/s]

preprocessing day:   3%|▎         | 1/31 [00:04<02:12,  4.40s/it]

preprocessing day:   6%|▋         | 2/31 [00:08<02:03,  4.26s/it]

preprocessing day:  10%|▉         | 3/31 [00:13<02:02,  4.36s/it]

preprocessing day:  13%|█▎        | 4/31 [00:17<01:53,  4.20s/it]

preprocessing day:  16%|█▌        | 5/31 [00:21<01:47,  4.15s/it]

preprocessing day:  19%|█▉        | 6/31 [00:25<01:46,  4.24s/it]

preprocessing day:  23%|██▎       | 7/31 [00:25<01:10,  2.94s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a031c317b20>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/26c3ea6875ccc62ab781d663441170aa/element


preprocessing day:  35%|███▌      | 11/31 [00:25<00:20,  1.04s/it]WAR

In [7]:
res

['https://n.news.naver.com/mnews/article/003/0009640953?sid=102',
 'https://n.news.naver.com/mnews/article/421/0004394383?sid=102',
 'https://n.news.naver.com/mnews/article/018/0004549362?sid=104',
 'https://n.news.naver.com/mnews/article/025/0002965713?sid=102',
 'https://n.news.naver.com/mnews/article/015/0004269147?sid=106',
 'https://n.news.naver.com/mnews/article/417/0000489994?sid=106',
 'https://n.news.naver.com/mnews/article/009/0004495421?sid=106',
 'https://n.news.naver.com/mnews/article/057/0001409472?sid=106',
 'https://n.news.naver.com/mnews/article/421/0004393497?sid=102',
 'https://n.news.naver.com/mnews/article/056/0010779813?sid=100',
 'https://n.news.naver.com/mnews/article/055/0000783482?sid=102',
 'https://n.news.naver.com/mnews/article/052/0001385061?sid=106',
 'https://n.news.naver.com/mnews/article/047/0002252368?sid=103']

In [6]:
naver_news_link

[['https://n.news.naver.com/mnews/article/001/0011309413?sid=102',
  'https://n.news.naver.com/mnews/article/023/0003497372?sid=110',
  'https://n.news.naver.com/mnews/article/421/0004385981?sid=101',
  'https://n.news.naver.com/mnews/article/417/0000488359?sid=101',
  'https://n.news.naver.com/mnews/article/023/0003497374?sid=110',
  'https://n.news.naver.com/mnews/article/023/0003497376?sid=110'],
 ['https://n.news.naver.com/mnews/article/421/0004388509?sid=102',
  'https://n.news.naver.com/mnews/article/055/0000782596?sid=102',
  'https://n.news.naver.com/mnews/article/003/0009635521?sid=101',
  'https://n.news.naver.com/mnews/article/056/0010778660?sid=102',
  'https://n.news.naver.com/mnews/article/028/0002480290?sid=103',
  'https://n.news.naver.com/mnews/article/214/0001004517?sid=101'],
 ['https://n.news.naver.com/mnews/article/055/0000782841?sid=102',
  'https://n.news.naver.com/mnews/article/020/0003262289?sid=105',
  'https://n.news.naver.com/mnews/article/055/0000782926?sid

In [8]:
from bs4 import BeautifulSoup
from selenium.common.exceptions import UnexpectedAlertPresentException

naver_news_title = []
naver_news_content = []

for n in tqdm(range(len(naver_news_link)),desc='preprocessing'):
    news_page_title = []
    news_page_content = []

    for idx in range(len(naver_news_link[n])):

        ######## 긁어온 URL로 접속하기 ########
        try:
            driver.get(naver_news_link[n][idx])
        except:
            print("Time Out!")
            continue

        ######## 접속한 URL에서 page_source받아오기 ########
        try:
            response = driver.page_source
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().alert()
            print("게시글이 삭제된 경우입니다.")
            continue
        soup = BeautifulSoup(response,'html.parser')

        ##### 뉴스 타이틀 긇어오기 #####
        title = None
        try:
            og_title = soup.find('meta', property='og:title')
            title = og_title['content']
        except:
            title = "OUT_LINK"

        news_page_title.append(title)

        ###### 뉴스 본문 긇어오기 #####
        doc = None
        text = ''

        data = soup.find_all("div",{"class":"newsct_article _article_body"})
        if data:
            for item in data:
                if item.find('script',type='text/javascript'):
                    item.find('script',type='text/javascript').decompose()
                elif item.find('span','end_photo_org'):
                    item.find('span','end_photo_org').decompose()
                text = item.get_text()
                doc = ''.join(text)
        else:
            doc = "OUT_LINK"

        news_page_content.append(doc.replace('\n',' '))

    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)

preprocessing:   0%|          | 0/6 [00:00<?, ?it/s]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a031c0cf580>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/26c3ea6875ccc62ab781d663441170aa/url


Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!


preprocessing:  17%|█▋        | 1/6 [00:02<00:10,  2.06s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a02fb215c90>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/26c3ea6875ccc62ab781d663441170aa/url


Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!


preprocessing:  33%|███▎      | 2/6 [00:04<00:08,  2.10s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a02fb215630>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/26c3ea6875ccc62ab781d663441170aa/url


...

preprocessing:  67%|██████▋   | 4/6 [00:08<00:04,  2.05s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a02fb2172e0>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/26c3ea6875ccc62ab781d663441170aa/url


Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!


preprocessing:  83%|████████▎ | 5/6 [00:10<00:02,  2.07s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a02fb215ae0>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/26c3ea6875ccc62ab781d663441170aa/url


Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!
Time Out!


preprocessing: 100%|██████████| 6/6 [00:12<00:00,  2.08s/it]


In [9]:
# 2020년 1월 1일 뉴스
naver_news_title[0]

[]

In [10]:
# 2020년 1월 8일 뉴스
naver_news_title[7]

IndexError: list index out of range

In [ ]:
naver_news_content[7][1]

'  질병관리본부 "격리치료 및 검사…현재 상태 양호"\'전파 증거 없다\'는 중국 발표 따라 \'관심단계\' 유지[세종=뉴시스] 임재희 기자 = 질병관리본부는 중국 우한시를 방문했다가 지난달 30일 입국한 중국 국적의 36세 여성 1명이 원인불명 폐렴 증상을 보여 격리 치료 중이라고 8일 밝혔다.질병관리본부 관계자는 "조사대상 유증상자로 분류해 격리 치료 및 검사를 실시 중"이라며 "상기 환자는 화난 해산물시장 방문이나 야생동물 접촉은 없었고 현재 상태는 양호하다"고 말했다.환자는 업무차 지난달 13~17일 중국 우한시를 방문했으며 한국에는 같은달 30일 입국했다. 기초 역학조사 결과 다음날인 31일부터 기침, 목 붓는 증상이 처음 나타났고 이달 7일 한림대학교 동탄성심병원에서 진료 중 우한시 방문력과 폐렴 소견이 확인, 질병관리본부에 신고됐다.중국 우한시 위생건강위원회가 지난달 31일 원인불명 폐렴 사실을 최초 발표한 이후 이달 5일 현재까지 59명의 환자가 보고됐다.질병관리본부는 우한시 방문력과 흉부방사선검사상 폐렴 소견을 근거로 이번 사례를 \'조사대상 유증상자\'로 분류했고 환자를 국가지정입원 치료병상이 있는 분당서울대병원으로 이송해 격리 상태로 치료와 검사를 했다. 현재 중앙 및 경기도 역학조사관이 역학조사를 진행 중이다.해당 환자는 이달 2~3일 기침과 발열로 오산한국병원에서 흉부방사선 검사 정상 판정에 따라 감기약 처방을 받았으며 6일에는 동탄성심병원에서 흉부방사선 검사 정상, 계절인플루엔자 검사 음성 판정을 받은 것으로 확인됐다. 해당 의료인에 대해선 진료 업무 배제 및 진료 공간 폐쇄 등 조치는 하지 않은 상태다.경기도 보건환경연구원이 시행한 호흡기바이러스 9종 검사결과는 음성이었으며 질병관리본부가 추가적으로 폐렴 유발 원인병원체에 대한 검사를 진행 중이다.메르스, 인플루엔자, 파라인플루엔자, 아데노바이러스, 사람보카바이러스, 호흡기세포융합바이러스, 리노바이러스, 사람메타뉴모바이러스, 사람코로나바이러스 등은 아니라는 얘기다.동반 여행자 및 접

In [ ]:
import pickle

# 크롤링된 결과 저장
with open("data\kr_naver_news_title.pk",'wb') as f:
    pickle.dump(naver_news_title,f)
with open("data\kr_naver_news_content.pk",'wb') as f:
    pickle.dump(naver_news_content,f)